In [1]:
import os
import csv
import glob
import json
import os
import glob
import numpy as np
import pandas as pd
import math
from scipy.signal import argrelmax
import matplotlib.pyplot as plt
from scipy import signal
import copy
import sklearn
import pickle

def CCC(x,y):

    ##正規化
    x=x-np.average(x)##平均値を引く
    x=x/np.std(x)##標準偏差で割る
    y=y-np.average(y)##平均値を引く
    y=y/np.std(y)##標準偏差で割る

    ##正規化相互相関を求め、その最大値を返す
#     print(np.argmax(np.correlate(x,y, "full")/len(x)),"/",len(np.correlate(x,y, "full")/len(x)))

    return max(np.correlate(x,y, "full")/len(x))

def TO_detect(data):###data:つま先骨盤感の距離
    TO=np.argmax(data)
    if TO>len(data)*0.7:###歩行周期の70％以降はエラー再検証
        TO=np.argmax(data[:int(len(data)*0.75)])
    return TO

def Lp(x,cut=10): #cut=10にしてるので，何も引数指定してなければ10になる．
    dt = 0.01                       # サンプリング間隔
    fn = 1/(2*dt)
    fp =cut                         # 通過域端周波数[Hz]
    fs =fp*1.5                      # 阻止域端周波数[Hz]
    gpass = 3                       # 通過域最大損失量[dB]
    gstop = 12                      # 阻止域最小減衰量[dB]
    # 正規化
    Wp = fp/fn
    Ws = fs/fn
    N, Wn = signal.buttord(Wp, Ws, gpass, gstop)#バターワースフィルタ作成
    b1, a1 = signal.butter(N, Wn, "low")#バターワースフィルタ作成
    y1 = signal.filtfilt(b1, a1, x)#バターワースフィルタ実行
    return y1

def time_parameter_detection(a):#a:かかとと骨盤の距離の時系列データ

    seikeiyou=a[int(len(a)/10):int(9*len(a)/10)]###平均値や標準偏差を求めるために中央の80％のみ抜き出す


    a=a-np.average(seikeiyou)##平均値を引く
    a=2*a/(max(seikeiyou)-min(seikeiyou))##標準偏差で割る

    ####加速度計算############################
    acc_a=[]###微分値(加速度を入れる)

    for i in range(len(a)-1):
        acc_a.append(abs(a[i]-a[i+1]))
    ###########################################


    ##########加速度や位置が異常値歩き終わりのご反応なのでカット#################
    while abs(max(acc_a[-100:-1]))>max(acc_a[0:int(3*len(acc_a)/4)])+0.05:
        acc_a=acc_a[:-1]
        a=a[:-1]
    while abs(min(acc_a[-100:-1]))>max(acc_a[0:int(3*len(acc_a)/4)])+0.05:
        acc_a=acc_a[:-1]
        a=a[:-1]
    while abs(min(a[-100:-1]))>1.3:
        acc_a=acc_a[:-1]
        a=a[:-1]

    #####################################################################


    #########しきい値以上から以下になる点を使って大まかに位置歩行周期ごとに区切る###################
    devitedtime=[]###大まかな区切り点を入れる
    for i in range(len(a)-10):
        if a[i]>-0.5 and a[i+1]<-0.5 and a[i+10]<-0.5:
            devitedtime.append(i)

    ##################################################################################

    ######大まかに区切った中での最小値を取る瞬間をかかと接地の瞬間として記録###############################
    detected_second=[]###かかと接地の瞬間
    for i in range(len(devitedtime)+1):
        if i==0 :
            if devitedtime[0]!=0:
                data=a[:devitedtime[0]]
                start_gait=0
            else:
                continue
        elif i==len(devitedtime):
            data=a[devitedtime[len(devitedtime)-1]:]
            start_gait=devitedtime[len(devitedtime)-1]
        else:
            data=a[devitedtime[i-1]:devitedtime[i]]
            start_gait=devitedtime[i-1]

        second = np.argmin(data)


        second=second+start_gait
        detected_second.append(second)
    ##################################################################################################


    ###############ここから下は誤検出の削除#################

    #############-1~1に変換しているのにかかと接地の瞬間の値がtarget以上はおかしいから削除#######################
    target=-0.5
    remove_list=[]
    for i in range(len(detected_second)-1):
        if a[detected_second[i]]>target:
            remove_list.append(detected_second[i])
    for i in remove_list:
        detected_second.remove(i)
    #############################################################################################


    #############ある歩行周期が極端に短くて次の周期がそれと同様の長さではない場合＝ご検出削除################
    for j in range(2):
        remove_list=[]
        for i in range(len(detected_second)-2):
            if detected_second[i+1]-detected_second[i]<80:
                if i==0:
                    if detected_second[i+2]-detected_second[i+1]>80: ####変更０２１８
                        remove_list.append(detected_second[i])
                        break
                    elif detected_second[i+2]-detected_second[i]<190:
                        remove_list.append(detected_second[i+1])
                        break


                if detected_second[i+2]-detected_second[i]<190:
                    remove_list.append(detected_second[i+1])
                    break
        for i in remove_list:
            detected_second.remove(i)
    for j in range(2):
        remove_list=[]
        for i in range(len(detected_second)-2):
            if detected_second[i+1]-detected_second[i]<80:
                if i==0:
                    remove_list.append(detected_second[i])
                    break
                if detected_second[i+2]-detected_second[i]<190:
                    remove_list.append(detected_second[i+1])
                    break
        for i in remove_list:
            detected_second.remove(i)
    ################################################################################################


    ####検出できたかかと接地の瞬間のかかと骨盤の距離が平均から大きく外れるものを削除#####################
    if len(detected_second)>3:
        len_list=[]
        for i in range(len(detected_second)):
            len_list.append(a[detected_second[i]])
        remove_list=[]
        for i in range(len(detected_second)):
            if len_list[i]-sum(len_list)/len(len_list)>30:
                remove_list.append(detected_second[i])
        for i in remove_list:
            detected_second.remove(i)
    ####################################################################################

    ####OpenPosenによる計測開始直後を削除######################
    if len(detected_second)>3:
        for i in range(len(detected_second)):

            if detected_second[0]<20:
                detected_second.remove(detected_second[0])
            else:
                break

    #############################################################################


    ###########最後と最初の歩行周期中のかかと骨盤の距離の波形が大きく異なる場合は削除
    if len(detected_second)>2:
        tar1=signal.resample(a[detected_second[0]:detected_second[1]],101)
        tar2=signal.resample(a[detected_second[-2]:detected_second[-1]],101)

        if CCC(tar1,tar2)<0.8:
            detected_second.remove(detected_second[-1])
    #####################################################################

    ####OpenPosenによる計測開始直後を削除を削除######################
    if detected_second[0]==0:
        detected_second.remove(detected_second[0])
    #############################################################################


    ####検出できたかかと接地の瞬間のかかと骨盤の距離が1歩行周期前から大きく外れるものを削除(歩きが遅い人のみ)#####################
    remove_list=[]

    if len(a)>1000:
        for i in range(len(detected_second)):
            if a[detected_second[i]-1]-a[detected_second[i]]>0.02:
                remove_list.append(detected_second[i])
        for i in remove_list:
            detected_second.remove(i)
    ######################################################################################################

    ####OpenPosenによる計測開終了直前を削除######################
    if abs(detected_second[-1]-len(a))<30:
        detected_second.remove(detected_second[-1])
    #####################################################
    return detected_second


###一歩行周期を荷重応答,単脚支持+前遊脚,遊脚に分割する
def four_devide(a,TO_second,TO_second_other):#a:位置歩行周期のデータ TO_second:観察側の脚のつま先離地,TO_second_other:反対側の脚のつま先離地
    a1=a[0:int(TO_second_other)]
    a2=a[int(TO_second_other):int(TO_second)]
    a4=a[TO_second_other:]

    print(TO_second,TO_second_other)

    return a1,a2,a4

###four_devideで4つに区切ったものの最大最小を抽出
def make_data(a,b,c):
    return[max(a),max(b),max(c),min(a),min(b),min(c)]

###four_devideで4つに区切ったものの最大最小を抽出その後値の範囲で割る
def make_data_perc(ran,a,b,c):
    return[max(a)/ran,max(b)/ran,max(c)/ran,min(a)/ran,min(b)/ran,min(c)/ran]


c:\Users\HYPERION\anaconda3\envs\TF\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\HYPERION\anaconda3\envs\TF\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\HYPERION\anaconda3\envs\TF\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [6]:
another_side= [6,8,9,12,14,15,17,19,25,26,31,32,34,35,36,37,39,40]####右側麻痺の被験者
number_of_subject=1#総数を入れる


###つま先接地かかかと接地どちらであるか予測 H_or_Fに代入##########
os.chdir("C:/Users/HYPERION/Desktop/ohp_data/")
with open('model_big.pickle', mode='rb') as f:
    clf = pickle.load(f)
os.chdir("C:/Users/HYPERION/Desktop/ohp_data/症例/")
filename="基本"+"*"
target=glob.glob(filename)
df=pd.read_excel(target[0],header=4,usecols= list(range(number_of_subject+1)),skipfooter=41)

name=df["sub No."].values
df=df.drop(["sub No."],axis=1)
df=df.T
df.columns =name
df=df[['SIAS-M. hip','SIAS-M. knee','SIAS-M. ankle']]
H_or_F=clf.predict(np.array(df.values))
#################################################################

##########健常者の関節の動き入手####################################
os.chdir("C:/Users/HYPERION/Desktop/ohp_data/")
teacher_KNEE=pd.read_csv("knee_teacher.csv").iloc[:,1].tolist()
teacher_hip=pd.read_csv("hip_teacher.csv").iloc[:,1].tolist()
teacher_ankle=pd.read_csv("ankle_teacher.csv").iloc[0,1:].tolist()
####################################################################

sub=list(range(1,number_of_subject+1))###被験者番号一覧
AFO_kind="no-AFO"###処理する制動力条件
dfdata_all=[]###最終的なデータセットの入れ物
all_gait_each_sub=[]###計測できた歩行周期数を記録



for sub_number in sub:

    ### 処理後のjasonファイルのある場所に移動
    place="C:/Users/HYPERION/Desktop/ohp_data/result2/sub-"+str(sub_number)+"/sub-"+str(sub_number)+"_"+AFO_kind
    os.chdir(place)
    ########################################

    #####もし2試行めがない場合できてしまった余計なファイル削除
    place2=place+"/"+str(2)
    try:
        os.rmdir(place2)
    except:
        dummy=0
    ##################################

    cycle=glob.glob("*")###この条件で何試行したのかを取得




    start_second=0 ###動画に人が写った最初の時間
    end_second=1400 ###動画に人が写った最後の時間

    gait_each_sub=0###歩行周期をカウント1始まり
    cycle_count=1###歩行周期をカウント1始まり
    for cycle_number in cycle:

        #####人が写っていないときはjson_loadでエラーが出るのでエラーから動画に人が写った最初の時間と動画に人が写った最後の時間を検出
        place="C:/Users/HYPERION/Desktop/ohp_data/result2/sub-"+str(sub_number)+"/sub-"+str(sub_number)+"_"+AFO_kind
        os.chdir(place)
        jason_count=-1
        jason_count_list=[]
        jason_count_list_post=[]
        jason_count_list_pre=[]
        print(place)
        os.chdir(cycle_number)
        datatype=["x","y","w"]*25
        dfl= pd.DataFrame(datatype,columns=["jason"])
        jsonnameR=glob.glob('*.json')##全部のjsonファイル名を取得
        jsonname=sorted(jsonnameR)###並び替えて正しい順番に
        for name in jsonname:
            json_open = open(name, 'r')
            json_load = json.load(json_open)
            jason_count=jason_count+1
            try:
                 data=json_load['people'][0]['pose_keypoints_2d']
            except:
                jason_count_list.append(jason_count)


        if jason_count_list!=0:
            for i in range (len(jason_count_list)-1):
                if jason_count_list[i+1]-jason_count_list[i]>200:
                    end_second=jason_count_list[i+1]-1
                    start_second=jason_count_list[i]+1
                    break
                if i == len(jason_count_list)-2:

                    if  jason_count_list[0]<10:
                        start_second=max(jason_count_list)+1
                        end_second=1450


                    else:
                        start_second=0
                        end_second=min((jason_count_list))-1




        else:
            print("##############################################error")
            end_second=1000
            start_second=0

        ##################################################################################################################


        t=0

        ###ssub=17 spring1は上のプログラムだと余計な部分も入ってしまうので手入力##################
        if sub_number==17 and AFO_kind=="spring1":
            start_second=start_second+100
        jsonnameR=glob.glob('*.json')##全部のjsonファイル名を取得
        jsonname=sorted(jsonnameR)[start_second:end_second-1]###並び替えて正しい順番に
        ##############################################################################


        ######とりあえず人jasonデータをCSVにまとめる(確認用#################)
        for name in jsonname:
            json_open = open(name, 'r')
            json_load = json.load(json_open)

            data=json_load['people'][0]['pose_keypoints_2d']
            df= pd.DataFrame(data,columns=[t])
            dfl=pd.concat([dfl,df], axis=1)
            t=t+0.01



        json_open.close()

        dfl.to_csv("元データ.csv", mode='w')
        ##########################################################################


        ####全データから必要なキーポイント抜き出し#######################################
        numberlist=[1,8,9,10,11,19,21,22,24,23,0,13,14,12,20]####使用するキーポイントの番号　この番号が何を示すかはOpenPoseのページで

        xlist=[] ###1,8,9,10,11,19,21,22,24 のxざひょう
        ylist=[] ##1,8,9,10,11,19,21,22,24 のyざひょう

        dfmc = pd.read_csv("元データ.csv", engine="python")
        dfmc=dfmc.replace(0, np.nan)
        for i in numberlist:
            xdata=dfmc.iloc[3*i,:]
            xdata= xdata.values.tolist()
            xlist.append(xdata)

            ydata=dfmc.iloc[3*i+1,:]
            ydata= ydata.values.tolist()
            ylist.append(ydata)
        ####################################################################################


        nan_front_list=[]

        for key in range(len(xlist)):
            for i in range (2,60):
                if np.isnan(xlist[key][i]) :
                    nan_front_list.append(i)



        if len(nan_front_list)!=0:
            while np.isnan(xlist[0][max(nan_front_list)+1]) or  np.isnan(xlist[1][max(nan_front_list)+1]) or np.isnan(xlist[2][max(nan_front_list)+1]) or np.isnan(xlist[3][max(nan_front_list)+1]) or np.isnan(xlist[4][max(nan_front_list)+1]) or np.isnan(xlist[5][max(nan_front_list)+1]) or np.isnan(xlist[6][max(nan_front_list)+1]) or np.isnan(xlist[7][max(nan_front_list)+1]) or np.isnan(xlist[8][max(nan_front_list)+1]) or np.isnan(xlist[9][max(nan_front_list)+1]) or np.isnan(xlist[10][max(nan_front_list)+1]) or np.isnan(xlist[11][max(nan_front_list)+1]) or np.isnan(xlist[12][max(nan_front_list)+1]) or np.isnan(xlist[13][max(nan_front_list)+1]) :
                nan_front_list.append(max(nan_front_list)+1)



        nan_back_list=[]
        for key in range(len(xlist)):
            for i in range (1,int(len(xlist[key])/10)):
                if np.isnan(xlist[key][-i]):
                    nan_back_list.append(i)

        if len(nan_back_list)!=0:
            while np.isnan(xlist[0][-max(nan_back_list)-1]) or  np.isnan(xlist[1][-max(nan_back_list)-1]) or np.isnan(xlist[2][-max(nan_back_list)-1]) or np.isnan(xlist[3][-max(nan_back_list)-1]) or np.isnan(xlist[4][-max(nan_back_list)-1]) or np.isnan(xlist[5][-max(nan_back_list)-1]) or np.isnan(xlist[6][-max(nan_back_list)-1]) or np.isnan(xlist[7][-max(nan_back_list)-1]) or np.isnan(xlist[8][-max(nan_back_list)-1]) or np.isnan(xlist[9][-max(nan_back_list)-1]) or np.isnan(xlist[10][-max(nan_back_list)-1]) or np.isnan(xlist[11][-max(nan_back_list)-1]) or np.isnan(xlist[12][-max(nan_back_list)-1]) or np.isnan(xlist[13][-max(nan_back_list)-1]) :
                nan_back_list.append(max(nan_back_list)+1)

        if len(nan_front_list)!=0:
            nan_front=max(nan_front_list)
        else:
            nan_front=2

        if len(nan_back_list)!=0:
            nan_back=max(nan_back_list)
        else:
            nan_back=0

        for i in range(len(xlist)):
            xlist[i]=xlist[i][nan_front+1:-nan_back-1]
            ylist[i]=ylist[i][nan_front+1:-nan_back-1]
        start_second=start_second+nan_front+1
        end_second=end_second-nan_back-1


        for i in range(len(xlist)):
            for j in range(2,42):
                if abs(xlist[i][j+1]-xlist[i][j])>100:
                    xlist[i][j+1]=np.nan


        t=0
        timerec=[]
        for i in range (len(xlist[0])-2):
            timerec.append(t)
            t=t+0.01
        dfl= pd.DataFrame(timerec,columns=["t"])


        for i in range (len(xlist)):
            df= pd.DataFrame(xlist[i],columns=[numberlist[i]])
            dfl=pd.concat([dfl,df], axis=1)


        df=dfl.interpolate('spline', order=3)


        xlist=[]
        for i in range(len(numberlist)):
            xdata=df.iloc[:,i+1]
            xdata= xdata.values.tolist()
            xlist.append(xdata)



        t=0
        timerec=[]
        for i in range (len(ylist[0])-2):
            timerec.append(t)
            t=t+0.01
        dfl= pd.DataFrame(timerec,columns=["t"])


        for i in range (len(ylist)):
            df= pd.DataFrame(ylist[i],columns=[numberlist[i]])
            dfl=pd.concat([dfl,df], axis=1)
        df=dfl.interpolate('spline', order=3)

        ylist=[]
        for i in range(len(numberlist)):
            ydata=df.iloc[:,i+1]
            ydata= ydata.values.tolist()
            ylist.append(ydata)



        ####かかとの移動から左右の入れ替わりを検知
        low_lib_change=[]
        min_L=xlist[6][100]
        max_R=xlist[8][100]
        max_R_list=[xlist[8][100]]
        min_L_list=[xlist[6][100]]
        for i in range(100,len(xlist[8])-100):
            if sub_number in another_side:
                if xlist[8][i]-max_R<-30:
                    low_lib_change.append(i)
                else:
                    if xlist[8][i]-max_R<30:
                        if xlist[8][i+1]-xlist[8][i]>-5:
                            max_R=xlist[8][i]
                    max_R_list.append(max_R)

            else:
                if min_L-xlist[6][i]<-30:
                    low_lib_change.append(i)
                else:
                    if min_L-xlist[6][i]<30:
                        if xlist[6][i]-xlist[6][i+1]>-5 :
                            min_L=xlist[6][i]
                min_L_list.append(min_L)



        if len(low_lib_change)!=0:

            Rlist=[3,4,7,8,9]
            Llist=[11,12,5,6,14]

            xlist_temp=copy.deepcopy(xlist)
            ylist_temp=copy.deepcopy(ylist)

            for sec in low_lib_change:
                for j in range(len(Rlist)):
                    xlist_temp[Rlist[j]][sec]=xlist[Llist[j]][sec]
                    xlist_temp[Llist[j]][sec]=xlist[Rlist[j]][sec]
                    ylist_temp[Rlist[j]][sec]=ylist[Llist[j]][sec]
                    ylist_temp[Llist[j]][sec]=ylist[Rlist[j]][sec]

            ylist=ylist_temp
            xlist=xlist_temp

        for i in range(len(xlist)):
            xlist[i]=Lp(xlist[i],4)
            ylist[i]=Lp(ylist[i],4)

        lendataHC_R=[]###かかと接地検出用の骨盤のとかかとの距離データ

        KNEE_dis=[]###かかと接地検出用の骨盤のとかかとの距離データ

        for i in range(0,len(xlist[1])):
            if sub_number in another_side:
                xlen=xlist[1][i]-xlist[8][i]
                KNEE_dis.append((-xlist[11][i]+xlist[3][i]))

            else:
                KNEE_dis.append((-xlist[11][i]+xlist[3][i]))
                xlen=-xlist[1][i]+xlist[6][i]
            leng=xlen
            lendataHC_R.append(leng)





        HC_Rsecond=time_parameter_detection(lendataHC_R)###かかと接地検出

        ###不具合のある歩行周期を削除##################################
        if sub_number==26 and AFO_kind=="spring3":
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==23 and AFO_kind=="no-AFO":
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==20 and AFO_kind=="spring2":
            HC_Rsecond.remove(HC_Rsecond[-1])
        print(cycle_number)
        ###2023/01/08
        if sub_number==22 and AFO_kind=="spring1":
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==29 and AFO_kind=="spring1":
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==22 and AFO_kind=="spring4":
            HC_Rsecond.remove(HC_Rsecond[-1])
            HC_Rsecond.remove(HC_Rsecond[-1])

        if sub_number==27 and AFO_kind=="spring3":
            HC_Rsecond.remove(HC_Rsecond[0])

        if sub_number==17 and AFO_kind=="no-AFO" and cycle_number=="2":
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==16 and AFO_kind=="spring2" and cycle_number=="2":
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==9 and AFO_kind=="spring3" and cycle_number=="2":
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==25 and AFO_kind=="spring1" :
            HC_Rsecond.remove(HC_Rsecond[-1])


        ##2023/01/10
        if sub_number==35 and AFO_kind=="spring1" : ###記録できていない
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==25 and AFO_kind=="spring1" : ###記録できていない
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==3 and AFO_kind=="spring2" : ###記録できていない
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==3 and AFO_kind=="spring2" and cycle_number =="2": ###直せない左右の入れ替わり
            HC_Rsecond.remove(HC_Rsecond[-1])

        ###2023/01/13
        if sub_number==31 and AFO_kind=="no-AFO" : ###フレーム落ち
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==31 and AFO_kind=="spring1" : ###フレーム落ち
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==31 and AFO_kind=="spring2" : ###フレーム落ち
            HC_Rsecond.remove(HC_Rsecond[-1])
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==31 and AFO_kind=="spring4" : ###フレーム落ち
            HC_Rsecond.remove(HC_Rsecond[-1])

        ###2023/01/13　afternoon
        if sub_number==20 and AFO_kind=="spring2" : ##外旋ひどい
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==23 and AFO_kind=="spring1" and cycle_number =="1": ##ぶん回し+外旋ひどい
            HC_Rsecond.remove(HC_Rsecond[0])
        ###2023/01/14　
        if sub_number==29 and AFO_kind=="no-AFO" : ###一時停止した
            HC_Rsecond.remove(HC_Rsecond[-1])

        if sub_number==30 and AFO_kind=="no-AFO" and cycle_number =="1": ###外旋による誤差
            HC_Rsecond.remove(HC_Rsecond[0])

        if sub_number==24 and AFO_kind=="spring1" : ###撮影できていない
            HC_Rsecond.remove(HC_Rsecond[0])
        if sub_number==35 and AFO_kind=="spring3" : ###直せない左右の入れ替わり
            HC_Rsecond.remove(HC_Rsecond[-1])
            HC_Rsecond.remove(HC_Rsecond[-1])
        if sub_number==20 and AFO_kind=="no-AFO" : ###後ろの人が映る
            HC_Rsecond.remove(HC_Rsecond[-1])
            HC_Rsecond.remove(HC_Rsecond[-1])
        #########################################################################################################


        real_HC_Rsecond=copy.deepcopy(HC_Rsecond)###ヴィデオ時間換算でのかかと接地の瞬間

        if AFO_kind =="no-AFO":
            print("no-AFO")
            print("no-AFO")
            print("no-AFO")
            print("no-AFO")


            if H_or_F[sub_number-1]=="F":
                print(sub_number)
    #######################################################################
                for i in HC_Rsecond:
                    if i-30<0:
                        print("no time")
                        tmp_time_start=i
                        tmp_time_end=i+30

                    else:
                        tmp_time_start=i-10
                        tmp_time_end=i+30
                    HC_Rsecond[HC_Rsecond.index(i)]=np.argmax(KNEE_dis[tmp_time_start:tmp_time_end])+tmp_time_start
    #####################################################################


        real_HC_Rsecond=copy.deepcopy(HC_Rsecond)

        for i in range(len(HC_Rsecond)):
            real_HC_Rsecond[i]=start_second+HC_Rsecond[i]




        ###########################以下よりデータセットの作成#########################################
        TO_second_list=[]###つま先離地面検出用
        TO_second_other_list=[]###反対の足のつま先離地面検出用


        for j in range(len(HC_Rsecond)-1):
            start_HC=HC_Rsecond[j]
            end_HC=HC_Rsecond[j+1]
            len_for_TO=[]
            len_for_TO_otherside=[]
            len_for_HC_otherside=[]
            len_for_HC=[]

            datalist=[sub_number,cycle_count]
            dfname=["subject","cycle"]
            cycle_count=cycle_count+1
            ###不具合のある歩行周期を削除##################################
            if sub_number==37 and AFO_kind=="spring1" and j ==3: ##OpenPOse推定ミス
                continue

            if sub_number==37 and AFO_kind=="spring2" and j ==1: ##入れ替わりが直せない
                continue

            if sub_number==37 and AFO_kind=="spring2" and j ==2: ##OpenPOse推定ミス
                continue

            ################################################



            gait_each_sub=gait_each_sub+1
            for i in range(HC_Rsecond[j],HC_Rsecond[j+1]):

                if sub_number in another_side:
                    len_for_TO.append(xlist[1][i]-xlist[7][i])
                    len_for_HC.append(-xlist[1][i]+xlist[8][i])
                    len_for_HC_otherside.append(-xlist[1][i]+xlist[6][i])
                    len_for_TO_otherside.append(xlist[1][i]-xlist[5][i])
                else:
                    len_for_TO.append(-xlist[1][i]+xlist[5][i])
                    len_for_HC.append(xlist[1][i]-xlist[6][i])
                    len_for_HC_otherside.append(xlist[1][i]-xlist[8][i])
                    len_for_TO_otherside.append(-xlist[1][i]+xlist[7][i])




            TO_second_other=TO_detect(len_for_TO_otherside)###歩行周期中の反対側のつま先離地
            TO_minimum=np.argmin(len_for_TO_otherside)###歩行周期中の反対側のつま先離地の時間


            zurashi=max(int(TO_detect(len_for_TO_otherside)*1.8),30)###歩行周期外をどれだけ見るか
            len_for_TO_otherside=[]###歩行周期外のつま先離地用データ
            for i in range(HC_Rsecond[j],HC_Rsecond[j+1]+zurashi):
                if sub_number in another_side:
                    try:
                        len_for_TO_otherside.append(xlist[1][i]-xlist[5][i])
                    except:
                        dummy=0

                else:
                    try:
                        len_for_TO_otherside.append(-xlist[1][i]+xlist[7][i])
                    except:
                        dummy=0

            while max(len_for_TO_otherside[-10:])>100:
                len_for_TO_otherside=len_for_TO_otherside[:-10]


            HC_second_other=np.argmax(len_for_HC_otherside)###歩行周期中の反対側のかかと接地
            TO_second=TO_detect(len_for_TO) ###歩行周期中の反対側のつま先離地
            TO_second_other_2nd=TO_minimum+np.argmax(len_for_TO_otherside[TO_minimum:])###歩行周期外のつま先離地
            gait_len=HC_Rsecond[j+1]-HC_Rsecond[j]
            ####sw####
            datalist.append(end_HC-TO_second-start_HC)
            dfname.append("SW")
            datalist.append((end_HC-TO_second-start_HC)/gait_len)
            dfname.append("SW%")
            #####st#########
            datalist.append(TO_second)
            dfname.append("ST")
            datalist.append(TO_second/gait_len)
            dfname.append("ST%")
            ####sw_o####
            datalist.append(HC_second_other-TO_second_other)
            dfname.append("SW_O")
            datalist.append((HC_second_other-TO_second_other)/gait_len)
            dfname.append("SW_O%")
            ####st_o####
            datalist.append(TO_second_other_2nd-HC_second_other)
            dfname.append("ST_O")
            datalist.append((TO_second_other_2nd-HC_second_other)/gait_len)
            dfname.append("ST_O%")
            ###Raito####
            datalist.append((end_HC-TO_second-start_HC)/(HC_second_other-TO_second_other))
            dfname.append("SW_raito")
            datalist.append(TO_second/(TO_second_other_2nd-HC_second_other))
            dfname.append("ST_raito")


            ###T####
            datalist.append(HC_Rsecond[j+1]-HC_Rsecond[j])
            dfname.append("T")

            ###V###
            if sub_number in another_side:
                datalist.append(abs(xlist[1][HC_Rsecond[j+1]]-xlist[1][HC_Rsecond[j]])/(HC_Rsecond[j+1]-HC_Rsecond[j])/2.1)
                dfname.append("V")
                datalist.append(abs(xlist[8][start_HC]-xlist[8][end_HC])/210)
                dfname.append("STRIDE")
                datalist.append(abs(xlist[8][start_HC]-xlist[6][start_HC+HC_second_other])/210)
                dfname.append("Step")
                datalist.append(abs(xlist[6][start_HC+HC_second_other]-xlist[8][end_HC])/210)
                dfname.append("Step_other")
                datalist.append(abs(xlist[8][start_HC]-xlist[6][start_HC+HC_second_other])/abs(xlist[6][start_HC+HC_second_other]-xlist[8][end_HC]))
                dfname.append("Step_raito")

            else:
                datalist.append(abs(xlist[1][HC_Rsecond[j+1]]-xlist[1][HC_Rsecond[j]])/(HC_Rsecond[j+1]-HC_Rsecond[j])/2.1)
                dfname.append("V")
                datalist.append(abs(xlist[6][start_HC]-xlist[6][end_HC])/210)
                dfname.append("STRIDE")
                datalist.append(abs(xlist[6][start_HC]-xlist[8][start_HC+HC_second_other])/210)
                dfname.append("Step")
                datalist.append(abs(xlist[8][start_HC+HC_second_other]-xlist[6][end_HC])/210)
                dfname.append("Step_other")
                datalist.append(abs(xlist[6][start_HC]-xlist[8][start_HC+HC_second_other])/abs(xlist[8][start_HC+HC_second_other]-xlist[6][end_HC]))
                dfname.append("Step_raito")
            datalist.append(120/(end_HC-start_HC))
            dfname.append("cadence")
            print("TO_second",TO_second)
            print("HC_second_other",HC_second_other)
            print("TO_second_other",TO_second_other)

            deg1list=[]#deg1：股関節
            deg2list=[]#deg2:膝関節
            deg3list=[]#deg3:足関節
            taikan_deglist=[]#体幹角度
            degtla23list=[]#TLA
            side_V=[]#瞬間的速度

            for i in range(HC_Rsecond[j],HC_Rsecond[j+1]-1):
                side_V.append(abs(xlist[1][i]-xlist[1][i+1]))
            ####contact way
            datalist.append(H_or_F[sub_number-1])
            dfname.append("contact way")

            #####関節角度の計算　deg1：股関節 deg2:膝関節　deg3:足関節###############################
            for i in range(HC_Rsecond[j],HC_Rsecond[j+1]):
                if sub_number in another_side:

                    #キーポイントのX座標を定義
                    keyx1=xlist[0][i]
                    keyx8=xlist[1][i]
                    keyx9=xlist[2][i]
                    keyx10=xlist[3][i]
                    keyx11=xlist[4][i]
                    keyx22=xlist[7][i]
                    keyx24=xlist[8][i]
                    keyx23=xlist[9][i]

                    #キーポイントのy座標を定義
                    keyy1=ylist[0][i]
                    keyy8=ylist[1][i]
                    keyy9=ylist[2][i]
                    keyy10=ylist[3][i]
                    keyy11=ylist[4][i]
                    keyy22=ylist[7][i]
                    keyy24=ylist[8][i]
                    keyy23=ylist[9][i]
                    suisen=[0,1]
                    heikou=[-1,0]
                else:


            #キーポイントのX座標を定義
                    keyx1=xlist[0][i]
                    keyx8=xlist[1][i]
                    keyx9=xlist[13][i]
                    keyx10=xlist[11][i]
                    keyx11=xlist[12][i]
                    keyx22=xlist[5][i]
                    keyx24=xlist[6][i]
                    keyx23=xlist[14][i]

                    #キーポイントのy座標を定義
                    keyy1=ylist[0][i]
                    keyy8=ylist[1][i]
                    keyy9=ylist[13][i]
                    keyy10=ylist[11][i]
                    keyy11=ylist[12][i]
                    keyy22=ylist[5][i]
                    keyy24=ylist[6][i]
                    keyy23=ylist[14][i]
                    suisen=[0,1]
                    heikou=[1,0]

                #ベクトルの定義を定義
                taikan=np.array([keyx1-keyx8,keyy1-keyy8])
                momo=np.array([keyx10-keyx9,keyy10-keyy9])
                sune=np.array([keyx11-keyx10,keyy11-keyy10])
                ashiura=np.array([keyx24-keyx22,keyy24-keyy22])
                tla23=np.array([keyx23-keyx8,keyy23-keyy8])
                tla22=np.array([keyx22-keyx8,keyy22-keyy8])

                #ベクトルの大きさ
                taikanlen=np.linalg.norm(taikan)
                sunelen=np.linalg.norm(sune)
                momolen=np.linalg.norm(momo)
                ashiuralen=np.linalg.norm(ashiura)
                tla23len=np.linalg.norm(tla23)
                tla22len=np.linalg.norm(tla22)

                #内積を求める
                hip_inner=np.inner(momo,taikan)
                knee_inner=np.inner(momo,sune)
                ankle_inner=np.inner(ashiura,sune)
                taikan_inner=np.inner(suisen,taikan)
                opdeg55_inner=np.inner(suisen,sune)
                opdeg56_inner=np.inner(ashiura,heikou)
                tla23_inner=np.inner(tla23,heikou)
                tla22_inner=np.inner(tla22,heikou)


                #角度tla23
                degtla23= math.degrees(np.arccos(tla23_inner/tla23len))
                degtla23list.append(degtla23-90)




                ###体幹　求める
                taikankaku= math.degrees(np.arctan2(np.cross(heikou,taikan),np.dot(heikou,taikan)))
                if sub_number in another_side:
                    taikankaku=(taikankaku-90)
                else:
                    taikankaku=(-taikankaku-90)
                taikan_deglist.append(taikankaku)




                deg1= math.degrees(np.arctan2(np.cross(momo,taikan),np.dot(momo,taikan)))
                deg1=(180-deg1)
                if deg1 >=300:
                    deg1=deg1-360
                if sub_number in another_side:
                    deg1=-deg1

                deg1list.append(deg1)


                #角度2を求める


                deg2= math.degrees(np.arctan2(np.cross(momo,sune),np.dot(momo,sune)))
                if sub_number in another_side:
                    deg2list.append(deg2)
                else:
                    deg2list.append(-deg2)

                #角度3を求める

                deg3= math.degrees(np.arctan2(np.cross(ashiura,sune),np.dot(ashiura,sune)))

                if sub_number in another_side:
                    if deg3>90:
                        deg3=-360+deg3

                    deg3list.append(deg3+90)


                else:
                    if deg3<-90:
                        deg3=360+deg3

                    deg3list.append(-deg3+90)

            ##############################################################################################

            #############1歩行周期数での値から変数を作成#####################################################

            hip_range=max(deg1list)-min(deg1list)
            knee_range=max(deg2list)-min(deg2list)
            ankle_range=max(deg3list)-min(deg3list)
            side_range=max(side_V)-min(side_V)
            taikan_range=max(taikan_deglist)-min(taikan_deglist)
            tla23_range=max(degtla23list)-min(degtla23list)

#             plt.plot(deg1list)
#             plt.show()
#             plt.plot(deg2list)
#             plt.show()
#             plt.plot(deg3list)
#             plt.show()

            datalist.append(max(deg1list))
            dfname.append("hip_max")
            datalist.append(max(deg2list))
            dfname.append("knee_max")
            datalist.append(max(deg3list))
            dfname.append("ankle_max")
            datalist.append(max(side_V))
            dfname.append("V_max")
            datalist.append(max(taikan_deglist))
            dfname.append("taikan_max")
            datalist.append(max(degtla23list))
            dfname.append("tla23_max")


            ###min
            datalist.append(min(deg1list))
            dfname.append("hip_min")
            datalist.append(min(deg2list))
            dfname.append("knee_min")
            datalist.append(min(deg3list))
            dfname.append("ankle_min")
            datalist.append(min(side_V))
            dfname.append("V_min")
            datalist.append(min(taikan_deglist))
            dfname.append("taikan_min")
            datalist.append(min(degtla23list))
            dfname.append("tla23_min")


            ###HC
            datalist.append(deg1list[0])
            dfname.append("hip_HC")
            datalist.append(deg2list[0])
            dfname.append("knee_HC")
            datalist.append(deg3list[0])
            dfname.append("ankle_HC")
            datalist.append(side_V[0])
            dfname.append("V_HC")
            datalist.append(taikan_deglist[0])
            dfname.append("taikan_HC")
            datalist.append(degtla23list[0])
            dfname.append("tla23_HC")


            ####CCC
            datalist.append(CCC(teacher_hip,signal.resample(deg1list,101)))
            dfname.append("hip_CCC")
            datalist.append(CCC(teacher_KNEE,signal.resample(deg2list,101)))
            dfname.append("knee_CCC")
            datalist.append(CCC(teacher_ankle,signal.resample(deg3list,101)))
            dfname.append("ankle_CCC")




            ####range
            datalist.append(hip_range)
            dfname.append("hip_range")
            datalist.append(knee_range)
            dfname.append("knee_range")
            datalist.append(ankle_range)
            dfname.append("ankle_range")
            datalist.append(side_range)
            dfname.append("V_range")
            datalist.append(taikan_range)
            dfname.append("taikan_range")
            datalist.append(tla23_range)
            dfname.append("tla23_range")

            ###gait separate time
            datalist.append(len(deg1list[0:int(TO_second_other)]))
            dfname.append("Time_LR")
            datalist.append(len(deg1list[int(TO_second_other):int(TO_second)]))
            dfname.append("Time_SS+PS")
            datalist.append(len(deg1list[TO_second:]))
            dfname.append("Time_SW")


            ###gait separate time
            datalist.append(len(deg1list[0:int(TO_second_other)])/len(deg1list))
            dfname.append("Time_LR%")
            datalist.append(len(deg1list[int(TO_second_other):int(TO_second)])/len(deg1list))
            dfname.append("Time_SS+PS%")
            datalist.append(len(deg1list[TO_second:])/len(deg1list))
            dfname.append("Time_SW%")

            deg1_1,deg1_2,deg1_3=four_devide(deg1list,TO_second,TO_second_other)
            deg2_1,deg2_2,deg2_3=four_devide(deg2list,TO_second,TO_second_other)
            deg3_1,deg3_2,deg3_3=four_devide(deg3list,TO_second,TO_second_other)
            side_1,side_2,side_3=four_devide(side_V,TO_second,TO_second_other)
            taikan_1,taikan_2,taikan_3=four_devide(taikan_deglist,TO_second,TO_second_other)
            tla23_1,tla23_2,tla23_3=four_devide(degtla23list,TO_second,TO_second_other)



            datalist.extend(make_data(deg1_1,deg1_2,deg1_3))
            datalist.extend(make_data(deg2_1,deg2_2,deg2_3))
            datalist.extend(make_data(deg3_1,deg3_2,deg3_3))
            datalist.extend(make_data(side_1,side_2,side_3))
            datalist.extend(make_data(taikan_1,taikan_2,taikan_3))
            datalist.extend(make_data(tla23_1,tla23_2,tla23_3))

            datalist.extend(make_data_perc(hip_range,deg1_1,deg1_2,deg1_3))
            datalist.extend(make_data_perc(knee_range,deg2_1,deg2_2,deg2_3))
            datalist.extend(make_data_perc(ankle_range,deg3_1,deg3_2,deg3_3))
            datalist.extend(make_data_perc(side_range,side_1,side_2,side_3))
            datalist.extend(make_data_perc(taikan_range,taikan_1,taikan_2,taikan_3))
            datalist.extend(make_data_perc(tla23_range,tla23_1,tla23_2,tla23_3))
            ################################################################################

            ######基本情報追加###########################################################
            place="C:/Users/HYPERION/Desktop/ohp_data/症例/"
            os.chdir(place)
            df_INDEX=pd.read_excel(target[0],header=4,usecols= list(range(number_of_subject+1)),skipfooter=41)
            nane="sub"+str(sub_number)
            datalist.extend(df_INDEX[nane].to_list())
            ###############################################################################

            dfdata_all.append(datalist)###処理してきたデータをデータセットに追加
    os.chdir("..")
    all_gait_each_sub.append(gait_each_sub) ###カウントしてきた歩行周期数を記録


################関節角度でmake_dataとmake_data_percを使って作成した文の名前作成###################
deg=["hip","knee","ankle","V","taikan","tla_small"]
for i in deg:
    for j in ["max","min"]:
        for k in ["LR","SS＋PS","SW"]:
            tmp=i+"_"+j+"_"+str(k)
            dfname.append(tmp)

for i in deg:
    for j in ["max","min"]:
        for k in ["LR","SS＋PS","SW"]:
            tmp="perc_"+i+"_"+j+"_"+str(k)
            dfname.append(tmp)


######################データセットの保存#############################
dfname.extend(df_INDEX["sub No."].to_list())
df=pd.DataFrame(dfdata_all,columns=dfname)

import datetime
dt_now = datetime.datetime.now()


place="C:/Users/HYPERION/Desktop/ohp_data/dataset_{}_{}_{}_{}.csv".format(AFO_kind,dt_now.year,dt_now.month,dt_now.day)
df.to_csv(place,encoding="cp932")
pd.DataFrame([all_gait_each_sub,sub],["gait","subject"]).T.to_csv("{}_gait_eachsub.csv".format(AFO_kind),encoding="cp932")

c:\Users\HYPERION\anaconda3\envs\TF\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\HYPERION\anaconda3\envs\TF\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\HYPERION\anaconda3\envs\TF\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.1.1 when using version 1.2.2. This might lead to breaki

C:/Users/HYPERION/Desktop/ohp_data/result2/sub-1/sub-1_no-AFO
1
no-AFO
no-AFO
no-AFO
no-AFO
TO_second 96
HC_second_other 65
TO_second_other 25
96 25
96 25
96 25
96 25
96 25
96 25
TO_second 95
HC_second_other 67
TO_second_other 29
95 29
95 29
95 29
95 29
95 29
95 29
C:/Users/HYPERION/Desktop/ohp_data/result2/sub-1/sub-1_no-AFO
2
no-AFO
no-AFO
no-AFO
no-AFO
TO_second 87
HC_second_other 62
TO_second_other 23
87 23
87 23
87 23
87 23
87 23
87 23
TO_second 92
HC_second_other 63
TO_second_other 25
92 25
92 25
92 25
92 25
92 25
92 25
